### Retrieve State Usage Data

This notebook downloads water use data for a user specified state and year from the National Water Information System server, and translates these data into physical supply and use table (PSUT), using a preformatted PSUT template and value mapping table. 

Original data are from the USGS state water programs. Below is a sample URL for Louisiana:
`https://waterdata.usgs.gov/la/nwis/water_use?format=rdb&rdb_compression=value&wu_area=County&wu_year=2000%2C2005%2C2010&wu_county=ALL&wu_category=ALL&wu_county_nms=--ALL%2BCounties--&wu_category_nms=--ALL%2BCategories--`

##### Workflow:
* Get the state and year from the user
* Get the filenames for the PSUT template and value mapping table
* Construct the url and retrieve the data into a pandas data frame
* Melt/gather the usage columns into row values under the column name 'Group'
* Remove rows with no usage data (identified by not having "Mgal" in the 'Group' name)

#### Required 3rd party modules
The following may require installation before running this notebook. These can each be installed using `pip`
* Pandas: `pip install pandas`
* OpenPyXl: `pip install openpyxl`
---

#### Begin by specifying the 2-character state code and the year to process

In [1]:
#Specify the state and year to process
state = 'la' #Louisiana
year = 2005

#### Next, you need to specify the path to the template PSUT and the value mapping filenames

In [2]:
#Get the input filenames
templateFilename = '../Data/Templates/StatePSUT_Template.xlsx'
remapFilename = '../Data/RemapTables/StatePSUTLookup.csv'

#### The rest is automated...

In [3]:
#Import modules
import sys, os, urllib
from shutil import copyfile
import pandas as pd
from openpyxl import load_workbook

##### Output data will be named with the state code and year (e.g. `la_2010.csv`) and will be saved in the StateData subfolder of the Data directory. This folder will be created, if it does not exist already.

In [4]:
#Create the StateData folder, if not already present
outFolder = '..\\Data\\StateData'
if os.path.exists(outFolder) == False:
    os.mkdir(outFolder)

In [5]:
#Set the output filename, putting it in the output data folder
outFilename = outFolder + os.sep + '{0}_{1}.xlsx'.format(state,year)
print("Output will be written to " + outFilename)

Output will be written to ..\Data\StateData\la_2005.xlsx


In [6]:
#Copy the template to the output filename
copyfile(src=templateFilename,dst=outFilename)

In [7]:
#Get the remap table and load as a dataframe
dfRemap = pd.read_csv(remapFilename,dtype='str',index_col="Index")

In [8]:
#Set the output file location to the Data/State data folder (relative to this file's location)
outFN = outFolder + os.sep + '{0}_{1}.csv'.format(state,year)

* Here we construct the URL for data retreival and then pull the on-line data to an in-memory data frame named `dfRaw`.

In [9]:
#Set the data URL path and parameters and construct the url
path = 'https://waterdata.usgs.gov/{}/nwis/water_use?'.format(state)
values = {'format':'rdb',
         'rdb_compression':'value',
         'wu_area':'County',
         'wu_year': year,
         'wu_county':'ALL',
         'wu_county_nms':'--ALL+Counties--',
         'wu_category_nms':'--ALL+Categories--'
        }
url = path + urllib.urlencode(values)

In [10]:
#Pull data in using the URL and remove the 2nd row of headers
dfRaw = pd.read_table(url,comment='#',header=[0,1],na_values='-')
dfRaw.columns = dfRaw.columns.droplevel(level=1)

In [11]:
#CHECK: Display a sample of the retrieved data
dfRaw.head()

,state_cd,state_name,county_cd,county_nm,year,"Total Population total population of area, in thousands","Public Supply population served by groundwater, in thousands","Public Supply population served by surface water, in thousands","Public Supply total population served, in thousands","Public Supply self-supplied groundwater withdrawals, fresh, in Mgal/d",...,Hydroelectric Power total offstream surface-water withdrawals in Mgal/d,"Hydroelectric Power power generated by instream use, in gigawatt-hours","Hydroelectric Power power generated by offstream use, in gigawatt-hours","Hydroelectric Power total power generated, in gigawatt-hours",Hydroelectric Power number of instream facilities,Hydroelectric Power number of offstream facilities,Hydroelectric Power total number of facilities,"Wastewater Treatment returns by public wastewater facilities, in Mgal/d",Wastewater Treatment number of public wastewater facilities,"Wastewater Treatment reclaimed wastewater released by wastewater facilities, in Mgal/d"
0,22,Louisiana,1,Acadia Parish,2005,59.552,43.057,0.000,43.057,6.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22,Louisiana,3,Allen Parish,2005,25.270,22.136,0.000,22.136,3.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22,Louisiana,5,Ascension Parish,2005,90.501,24.909,22.874,47.783,2.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,Louisiana,7,Assumption Parish,2005,23.196,0.000,22.823,22.823,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22,Louisiana,9,Avoyelles Parish,2005,42.098,39.784,0.000,39.784,4.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### With the data now held locally, we reformat the table so that each of the usage (and other) columns are "[melted](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)" into a single column such that the original column name is stored in a new field and the value is stored in another. This facilitates subsequent analysis, which includes:
 * Removing rows (formerly columns) that don't report volume, e.g. population served values, and
 * Ensuring that volume data is a floating point number.

In [12]:
#Tidy the data: transform so data in each usage column become row values with a new column listing the usage type
rowHeadings = ['county_cd', 'county_nm', 'state_cd', 'state_name', 'year']
dfTidy = pd.melt(dfRaw,id_vars=rowHeadings,value_name='MGal',var_name='Group')
print("Data transformed from {0} rows/columns to {1} rows/columns".format(dfRaw.shape, dfTidy.shape))

Data transformed from (64, 281) rows/columns to (17664, 7) rows/columns


In [13]:
#Remove rows that don't have volume data (i.e. keep only columns with 'Mgal' in the name)
dfTidy = dfTidy[dfTidy['Group'].str.contains('Mgal')].copy(deep=True)
dfTidy.shape

(15040, 7)

In [14]:
#Change the type of the MGal column to float 
dfTidy['MGal'] = dfTidy.MGal.astype("float")

In [15]:
#CHECK: Show the structure of the 'tidied' data frame
dfTidy.head()

,county_cd,county_nm,state_cd,state_name,year,Group,MGal
256,1,Acadia Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,6.04
257,3,Allen Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,3.71
258,5,Ascension Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,2.64
259,7,Assumption Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,0.00
260,9,Avoyelles Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,4.60


##### Next, we join the value mapping table to the tidied data frame, thus linking the values (in MGal/year) with the address of the PSUT cell in which they should be ported. 

In [16]:
#Join the remap table
dfAll = pd.merge(dfTidy,dfRemap,how='inner',left_on="Group",right_on="Group")
dfAll.head()

,county_cd,county_nm,state_cd,state_name,year,Group,MGal,Column1,Row1,Column2,Row2,Column3,Row3
0,1,Acadia Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,6.04,N,31,NaN,NaN,NaN,NaN
1,3,Allen Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,3.71,N,31,NaN,NaN,NaN,NaN
2,5,Ascension Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,2.64,N,31,NaN,NaN,NaN,NaN
3,7,Assumption Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,0.00,N,31,NaN,NaN,NaN,NaN
4,9,Avoyelles Parish,22,Louisiana,2005,Public Supply self-supplied groundwater withdr...,4.60,N,31,NaN,NaN,NaN,NaN


In [ ]:
#Open the spreadsheet template
wb = load_workbook(filename=outFilename)
ws = wb['Template']
ws.title = str(year)

In [ ]:
#Loop through the first set of row/columns and insert values into the Excel spreadsheet
dfRound1 = dfAll.groupby(['Column1','Row1'])['MGal'].sum()
dfRound1.fillna(value="n/a",inplace=True)
for (row,column), value in dfRound1.iteritems():
    #Set the value in the workbook
    rv = str(row)+ str(column)
    ws[rv] = value
#Save the workbook
wb.save(outFilename)

In [ ]:
#Loop through the second set of row/columns and insert values into the Excel spreadsheet
dfRound2 = dfAll.groupby(['Column2','Row2'])['MGal'].sum()
dfRound2.fillna(value="n/a",inplace=True)
for (row,column), value in dfRound2.iteritems():
    #Set the value in the workbook
    rv = str(row)+ str(column)
    ws[rv] = value
#Save the workbook
wb.save(outFilename)

In [ ]:
#Loop through the third set of row/columns and insert values into the Excel spreadsheet
dfRound3 = dfAll.groupby(['Column3','Row3'])['MGal'].sum()
dfRound3.fillna(value="n/a",inplace=True)
for (row,column), value in dfRound3.iteritems():
    #print row, column, value
    #Set the value in the workbook
    rv = str(row)+ str(column)
    ws[rv] = value
#Save the workbook
wb.save(outFilename)